# Converting System Volt-Var Parameters to 1547 Tabular Format

This notebook illustrates the conversion of system-oriented volt-var function parameters, e.g., slope (gain) and deadband, into the standard table of V1..V4, Q1..Q4 points as defined in IEEE 1547-2018. 

Within the scope of IEEE 1547-2018, the table of points may also be shifted with a "reactive power bias level", *qbias*. This has the effect of implementing a constant Q mode, modified by a volt-var response to voltage deviations. This allows DER to participate in steady-state grid voltage control as a reactive power resource, dispatched like shunt capacitors or reactors, but still responding autonomously to local voltage excursions.

Run the following cell to define the plot and table functions using [Matplotlib](https://matplotlib.org/)

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math

# convert center, deadband, slope, and q limits into a table of V and Q points.
# the function returns two arrays for the V and Q points
#    the arrays have sentinel elements below V1 and above V4, so they are 6 elements long (not 4)
#    the sentinel elements clarify that constant extrapolation is used outside the range [V1..V4]
def set_characteristic (center=1.0, deadband=0.0, slope=22.0, qmax=0.44, qmin=-0.44, qbias=0.0):
  if qbias > qmax:
    qbias = qmax
  elif qbias < qmin:
    qbias = qmin
  Q1 = qmax
  Q2 = qbias
  Q3 = qbias
  Q4 = qmin
  V2 = center - 0.5 * deadband
  V3 = center + 0.5 * deadband
  V1 = V2 - (Q1 - Q2) / slope
  V4 = V3 - (Q4 - Q3) / slope
  VL = V1 - 0.01
  VH = V4 + 0.01
  vtable = np.array ([VL, V1, V2, V3, V4, VH])
  qtable = np.array ([Q1, Q1, Q2, Q3, Q4, Q4])
  return vtable, qtable

# this function plots and tabulates a volt-var characteristic
def show_characteristic (label, center, deadband, slope, qmax, qmin, qbias=0.0):
  vtable, qtable = set_characteristic (center, deadband, slope, qmax, qmin, qbias)
  
  # bounds for plotting the horizontal axis
  vmin = vtable[0]-0.01
  vmax = vtable[-1]+0.01
  
  # evaluate the characteristic over 500 equal voltage intervals
  v = np.linspace (vmin, vmax, 501)
  # interpolating Q using the numpy library function
  q = np.interp (v, vtable, qtable)
  
  # create the plot (subplots call optimized for PDF generation)
  fig, ax = plt.subplots(1, 1, figsize=(9,4), tight_layout=True)
  fig.suptitle ('{:s} volt-var characteristic'.format (label))
  
  ax.plot (vtable, qtable, marker='o', color='blue', label='Points and Sentinels')
  ax.plot (v, q, color='red', label='Interpolated')
  ax.grid ()
  ax.set_xlabel ('V [pu]')
  ax.set_ylabel ('Q [pu]')
  ax.set_xlim (vmin, vmax)
  ax.legend ()
  
  # create the data table with 3 columns
  cellText = []
  cellText.append (['INPUTS', '', ''])
  cellText.append (['center', '{:.3f}'.format (center), ''])
  cellText.append (['deadband', '{:.3f}'.format (deadband), ''])
  cellText.append (['slope', '{:.3f}'.format (slope), ''])
  cellText.append (['Qmax', '{:.3f}'.format (qmax), ''])
  cellText.append (['Qmin', '{:.3f}'.format (qmin), ''])
  cellText.append (['Qbias', '{:.3f}'.format (qbias), ''])
  cellText.append (['', '', ''])
  cellText.append (['TABLE', 'V', 'Q'])
  for i in range(4):
    cellText.append (['{:d}'.format(i+1), '{:.3f}'.format(vtable[i+1]), '{:.3f}'.format(qtable[i+1])])
  cwidth = 0.2
  plt.table (cellText=cellText, cellLoc='center', colWidths=[cwidth, cwidth, cwidth], loc='right')

  plt.show ()

# use the current directory as default location for the "save plot" buttons
# optimize the graphic export for LaTex and PDF
plt.rcParams['savefig.directory'] = os.getcwd()
plt.rcParams['savefig.pad_inches'] = 0.05
plt.rcParams['savefig.dpi'] = 300.0
plt.rcParams['savefig.bbox'] = 'tight'
# invoke the Jupyter support for  Matplotlib graphics
%matplotlib widget


## Examples

Run the following cell to show several volt-var characteristics of interest.

In [ ]:
show_characteristic ('Default category A',    center=1.0, deadband=0.0,  slope=2.5,      qmax=0.25, qmin=-0.25)
show_characteristic ('Aggressive category A', center=1.0, deadband=0.0,  slope=12.5,     qmax=0.25, qmin=-0.25)
show_characteristic ('Default category B',    center=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44)
show_characteristic ('Aggressive category B', center=1.0, deadband=0.0,  slope=22.0,     qmax=0.44, qmin=-0.44)
show_characteristic ('Hawaii Rule 14H',       center=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44)


## Examples with *Qbias*

Run the following cell to repeat the example volt-var characteristics, with positive (capacitive) and negative (inductive) bias levels for steady-state reactive power from the DER.

In [ ]:
show_characteristic ('Default Cat A, +Qbias', center=1.0, deadband=0.0,  slope=2.5, qmax=0.25, qmin=-0.25, qbias=0.1)
show_characteristic ('Default Cat A, -Qbias', center=1.0, deadband=0.0,  slope=2.5, qmax=0.25, qmin=-0.25, qbias=-0.1)

show_characteristic ('AARV Cat A, +Qbias', center=1.0, deadband=0.0,  slope=12.5,  qmax=0.25, qmin=-0.25, qbias=0.1)
show_characteristic ('AARV Cat A, -Qbias', center=1.0, deadband=0.0,  slope=12.5,  qmax=0.25, qmin=-0.25, qbias=-0.1)

show_characteristic ('Default Cat B, +Qbias', center=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('Default Cat B, -Qbias', center=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44, qbias=-0.1)

show_characteristic ('AARV Cat B, +Qbias', center=1.0, deadband=0.0,  slope=22.0,  qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('AARV Cat B, -Qbias', center=1.0, deadband=0.0,  slope=22.0,  qmax=0.44, qmin=-0.44, qbias=-0.1)

show_characteristic ('HI Rule 14H, +Qbias', center=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('HI Rule 14H, -Qbias', center=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44, qbias=-0.1)